DIMTIME Dimension Table

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import functions as f

import datetime

import numpy as np
import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession




In [0]:
minstartdate = spark.sql('''SELECT MIN(started_at) FROM trips''')
maxstartdate = spark.sql('''SELECT DATEADD(year, 5, MAX(started_at)) FROM trips''')

display(minstartdate)
display(maxstartdate)

min(started_at)
2021-02-01 01:07:04


"timestampadd(year, 5, max(started_at))"
2027-01-31T23:58:37.000+0000


Create dimtime dataframe

In [0]:
times = pd.date_range(start='2021-02-01 01:07:04', end='2027-02-01 01:07:04', freq = 'D')
dimtime = pd.DataFrame(times)
headers = ["time_id"]
dimtime.columns = headers
dimtime['adate'] = pd.DatetimeIndex(dimtime['time_id']).date
dimtime['ayear'] = pd.DatetimeIndex(dimtime['time_id']).year
dimtime['amonth'] = pd.DatetimeIndex(dimtime['time_id']).month
dimtime['aquarter'] = pd.DatetimeIndex(dimtime['time_id']).quarter
dimtime['adayofweek'] = pd.DatetimeIndex(dimtime['time_id']).dayofweek
dimtime['adayofmonth'] = pd.DatetimeIndex(dimtime['time_id']).day
dimtime = dimtime.drop(columns = ['time_id'])
dimtime.insert(0, 'time_id',range(1, 1+len(dimtime)))
dimtime.head()
dimtime=spark.createDataFrame(dimtime) 


Out[6]:

,time_id,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
0,1,2021-02-01,2021,2,1,0,1
1,2,2021-02-02,2021,2,1,1,2
2,3,2021-02-03,2021,2,1,2,3
3,4,2021-02-04,2021,2,1,3,4
4,5,2021-02-05,2021,2,1,4,5


Create dimtime table using dimtime dataframe in the databricks delta storage

In [0]:
dimtime.write.format("delta").mode("overwrite").saveAsTable("dimtime")

In [0]:
%sql
select * from dimtime limit 2;

time_id,adate,ayear,amonth,aquarter,adayofweek,adayofmonth
1,2021-02-01,2021,2,1,0,1
2,2021-02-02,2021,2,1,1,2


DIMSTATION Dimension Table

In [0]:
dimstation = spark.sql('''
SELECT station_id, name as stationname, latitude, longitude from stations
''')

In [0]:
dimstation.write.format("delta").mode("overwrite").saveAsTable("dimstation")

In [0]:
%sql
select * from dimstation limit 2;

station_id,stationname,latitude,longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.66605799999999
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668


DIMRIDER Dimension Table

In [0]:
dimrider = spark.sql('''
SELECT rider_id, firstname, lastname, address, birthday, is_member as ismember from riders
''')

In [0]:
dimrider.write.format("delta").mode("overwrite").saveAsTable("dimrider")

In [0]:
%sql
select * from dimrider limit 2;

rider_id,firstname,lastname,address,birthday,ismember
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13,True
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10,True


FACTPAYMENT Fact Table

In [0]:
factpayment = spark.sql('''
SELECT payment_id, dates, amount, rider_id from payments
''')

In [0]:
factpayment.write.format("delta").mode("overwrite").saveAsTable("factpayment")

In [0]:
%sql
select * from factpayment limit 2;

payment_id,dates,amount,rider_id
1,2019-05-01,9.0,1000
2,2019-06-01,9.0,1000


FACTTRIP Fact Table

In [0]:
facttrip = spark.sql('''
SELECT trips.trip_id, trips.rideable_type, trips.started_at as tripstarttime, trips.ended_at as tripendtime, trips.start_station_id, trips.end_station_id,
DATEDIFF(hour, trips.started_at, trips.ended_at) as duration,
DATEDIFF(year, riders.birthday, trips.started_at) as rider_age,
trips.rider_id
from trips join riders ON riders.rider_id = trips.rider_id 
join dimtime as dt1 ON dt1.adate = LEFT(trips.started_at,10)
join dimtime as dt2 ON dt2.adate = LEFT(trips.ended_at,10)
''')

In [0]:
facttrip.write.format("delta").mode("overwrite").saveAsTable("facttrip")

In [0]:
%sql
select * from facttrip limit 2;

trip_id,rideable_type,tripstarttime,tripendtime,start_station_id,end_station_id,duration,rider_age,rider_id
222BB8E5059252D7,classic_bike,2021-06-13 09:48:47,2021-06-13 10:07:23,KA1503000064,13021,0,30,34062
1826E16CB5486018,classic_bike,2021-06-21 22:59:13,2021-06-21 23:04:29,TA1306000010,13021,0,26,5342
